# 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [24]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [25]:
digits = datasets.load_digits()

x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=25)

clf = GradientBoostingRegressor(random_state=5)

clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

MSE = metrics.mean_squared_error(y_test, y_pred)
print("MSE:%.3f" % MSE)

MSE:1.493


In [26]:
# List all hyper-parameter which will be tuned.
import numpy as np

learning_rate = np.arange(0.1, 0.5, 0.1)
n_estimators = np.arange(100, 20, -20)
max_depth = np.arange(3, 12, 3)

param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   10.5s finished


In [27]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -1.480330 using {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 100}


In [28]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'],
                                         learning_rate=grid_result.best_params_['learning_rate'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

# 調整參數後約 MSE
print("MSE:%.3f" % metrics.mean_squared_error(y_test, y_pred))

MSE:1.084
